In [1]:
# Author: De Vo 
# Version: 1.0 
# Date: 12/10/2018

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, on=["school_name"])
#school_data_complete.head(3)

## District summary

In [2]:
total_schools = len(school_data['school_name'])
total_student = len(student_data['Student ID'])
total_budget = school_data['budget'].sum()
avg_math_score = round(student_data['math_score'].sum()/total_student,4)
avg_reading_score = round(student_data['reading_score'].sum()/total_student,4)
percent_pass_math = len(student_data[student_data['math_score'] >= 70])*100/total_student
percent_pass_reading = len(student_data[student_data['reading_score'] >= 70])*100/total_student
overall_passing = (percent_pass_math + percent_pass_reading)/2

#District Summary 
district_summary = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_student],
           "Total Budget": [total_budget], "Average Math Score":[avg_math_score],
           "Average Reading Score":[avg_reading_score], "% Passing Math":[percent_pass_math],
          "% Passing Reading":[percent_pass_reading], "% Overall Passing Rate":[overall_passing]
          })
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.9854,81.8778,74.980853,85.805463,80.393158


## School Summary

In [3]:
#groupby_school = school_data_complete.groupby(['school_name'])
school_name = school_data_complete['school_name'].unique()
school_summary = {}
for name in school_name:
    school_summary.update({name:[]}) #dictionary of all the school

for i in range(len(school_name)):
    data = school_data_complete[school_data_complete['school_name'] == school_name[i]] #get a subset of the data with that shcool name 
    
    #name = school_name[i]
    school_type = list(data['type'])[0]
    total_student = data['Student ID'].count()
    total_school_budget = list(data['budget'])[0]
    Per_Student_Budget = total_school_budget/total_student
    average_math_score = round(data['math_score'].sum()/total_student,4)
    average_reading_score = round(data['reading_score'].sum()/total_student,4)
    pass_math = round(len(data[data['math_score'] >= 70])*100/total_student,4)
    pass_reading = round(len(data[data['reading_score'] >= 70])*100/total_student,4)
    avg_passing_rate = round((pass_math + pass_reading)/2,4)
    school_summary[school_name[i]] = [school_type,total_student,total_school_budget,
                                      Per_Student_Budget,average_math_score,average_reading_score,
                                      pass_math,pass_reading,avg_passing_rate]

#create DataFrame
result = pd.DataFrame(school_summary)
label = [['School Type','Total Students','Total School Budget',
          'Per Student Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
result_t = result.set_index(label).T

## Top Performing Schools (By Passing Rate)

In [4]:
result_t.sort_values(by = ['% Overall Passing Rate'], ascending = False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582,83.0619,83.9758,94.1335,97.0398,95.5866
Thomas High School,Charter,1635,1043130,638,83.4183,83.8489,93.2722,97.3089,95.2906
Pena High School,Charter,962,585858,609,83.8399,84.0447,94.5946,95.9459,95.2702
Griffin High School,Charter,1468,917500,625,83.3515,83.8168,93.3924,97.139,95.2657
Wilson High School,Charter,2283,1319574,578,83.2742,83.9895,93.8677,96.5396,95.2036


## Bottom Performing Schools (By Passing Rate)

In [5]:
result_t.sort_values(by = ['% Overall Passing Rate']).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363,637,76.8427,80.7447,66.3666,80.2201,73.2934
Figueroa High School,District,2949,1884411,639,76.7118,81.158,65.9885,80.7392,73.3638
Huang High School,District,2917,1910635,655,76.6294,81.1827,65.6839,81.3164,73.5001
Johnson High School,District,4761,3094650,650,77.0725,80.9664,66.0576,81.2224,73.64
Ford High School,District,2739,1763916,644,77.1026,80.7463,68.3096,79.299,73.8043


## Math Scores by Grade

In [6]:
#get unique school name and store in school_grade 
grade = school_data_complete['grade'].unique()

#function to sort grade (small - large) - for display purposes 
def sort_grade(school_grade_list):
    arr=[]
    for grade in school_grade_list:
        temp = grade.replace('th',"")
        arr.append(int(temp))

    arr.sort()
    result =[]
    for number in arr:
        result.append(str(number)+'th')
    return result

#grade after sort
school_grade = sort_grade(grade)

#create an empty dataFrame and append in data
math_by_grade = pd.DataFrame(index = school_name, columns = school_grade) #empty DataFrame for Math 
reading_by_grade = pd.DataFrame(index = school_name, columns = school_grade) # empty DF for reading 

for school in school_name:
    for grade in school_grade:
        temp_df = school_data_complete[(school_data_complete['school_name'] == school) & (school_data_complete['grade']== grade)]
        
        temp_math = round(temp_df['math_score'].sum()/len(temp_df),2)
        temp_reading = round(temp_df['reading_score'].sum()/len(temp_df),2)
        
        math_by_grade.loc[school,grade] = temp_math #store value into DF 
        reading_by_grade.loc[school,grade] = temp_reading
        
math_by_grade #Display math by grade 

,9th,10th,11th,12th
Huang High School,77.03,75.91,76.45,77.23
Figueroa High School,76.4,76.54,76.88,77.15
Shelton High School,83.42,82.92,83.38,83.78
Hernandez High School,77.44,77.34,77.14,77.19
Griffin High School,82.04,84.23,83.84,83.36
Wilson High School,83.09,83.72,83.2,83.04
Cabrera High School,83.09,83.15,82.77,83.28
Bailey High School,77.08,77,77.52,76.49
Holden High School,83.79,83.43,85,82.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade

In [7]:
reading_by_grade #calculated same loop with Math score

,9th,10th,11th,12th
Huang High School,81.29,81.51,81.42,80.31
Figueroa High School,81.2,81.41,80.64,81.38
Shelton High School,84.12,83.44,84.37,82.78
Hernandez High School,80.87,80.66,81.4,80.86
Griffin High School,83.37,83.71,84.29,84.01
Wilson High School,83.94,84.02,83.76,84.32
Cabrera High School,83.68,84.25,83.79,84.29
Bailey High School,81.3,80.91,80.95,80.91
Holden High School,83.68,83.32,83.82,84.7
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

In [8]:
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
column_names = ["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]
spending_bins = [0, 585, 615, 645, 675]
score_by_spending = pd.DataFrame(index = group_names, columns = column_names)

for i in range(len(spending_bins)-1):
    for name in column_names:
        temp_dataframe = result_t[(result_t['Per Student Budget'] > spending_bins[i]) & (result_t['Per Student Budget'] < spending_bins[i+1])]
        temp_result = round(temp_dataframe[name].sum()/len(temp_dataframe),3)
        score_by_spending .loc[group_names[i],name] = temp_result
        
score_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
<$585,83.455,83.934,93.46,96.611,95.035
$585-615,83.6,83.885,94.231,95.9,95.066
$615-645,79.079,81.891,75.668,86.107,80.887
$645-675,76.997,81.028,66.165,81.134,73.649


## Scores by School Size

In [9]:
size_bins = [0, 1000, 2000, 5000]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
score_by_size = pd.DataFrame(index = size_names, columns = column_names)

for i in range(len(size_bins)-1):
    for name in column_names:
        temp_dataframe = result_t[(result_t['Total Students'] > size_bins[i]) & (result_t['Total Students'] < size_bins[i+1])]
        temp_result = round(temp_dataframe[name].sum()/len(temp_dataframe),3)
        score_by_size.loc[size_names[i],name] = temp_result
        
score_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Small (<1000),83.822,83.93,93.55,96.099,94.825
Medium (1000-2000),83.375,83.864,93.6,96.791,95.195
Large (2000-5000),77.746,81.345,69.963,82.767,76.365


## Scores by School Type

In [10]:
#size_bins = [0, 1000, 2000, 5000]
type_names = result_t['School Type'].unique()
score_by_size = pd.DataFrame(index = type_names, columns = column_names)
for i in range(len(type_names)):
    for name in column_names:
        temp_dataframe = result_t[result_t['School Type'] == type_names[i]]
        temp_result = round(temp_dataframe[name].sum()/len(temp_dataframe),3)
        score_by_size.loc[type_names[i],name] = temp_result
        
score_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
District,76.957,80.967,66.548,80.799,73.674
Charter,83.474,83.896,93.621,96.586,95.104
